# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = os.path.join("..","WeatherPy", "output_data", "city_weather_data.csv")
clean_city_weather_data_df = pd.read_csv(csvpath)
clean_city_weather_data_df


,City Name,City ID,Country,Latitude,Longitude,Date of Data Calculation,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Barcelona,3128760,ES,41.39,2.16,1593039780,75.00,83,20,4.70
1,Mana,3381041,GF,5.66,-53.78,1593039830,78.49,86,33,6.64
2,Rikitea,4030556,PF,-23.12,-134.97,1593039830,70.66,56,89,10.56
3,Wilmington,4499379,US,34.23,-77.94,1593039563,86.00,66,20,10.29
4,Puerto Ayora,3652764,EC,-0.74,-90.35,1593039830,71.01,90,48,3.00
...,...,...,...,...,...,...,...,...,...,...
545,Simbahan,1695180,PH,6.30,120.58,1593039896,83.16,72,94,4.12
546,Jinchang,1805733,CN,38.50,102.17,1593039896,70.41,20,82,3.71
547,Tessaoua,2438823,NE,13.76,7.99,1593039896,94.75,19,19,5.03
548,Pangnirtung,6096551,CA,66.15,-65.71,1593039897,39.20,93,75,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
#load locations and humity for each location
locations = clean_city_weather_data_df[["Latitude", "Longitude"]]
humidity = clean_city_weather_data_df["Humidity (%)"]

# Plot Heatmap
humidity_fig = gmaps.figure()

# Create heat layer and add to map
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

humidity_fig.add_layer(humidity_layer)
humidity_fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#narrow city list by max temp, humidity, cloudiness, wind speed, and country of interest
vaca_country = ['AR', 'AU', 'BR', 'BS', 'CH', 'CL', 'DO', 'EG', 'ES', 'FI', 'FR', 'GB', 'GL', 'IE', 'IS', 'IT', 'JP', 'MX', 'NO', 'NP', 'NZ', 'PF', 'SE', 'VG']
#reduce list to countires of interest
vaca_city_df = clean_city_weather_data_df.query("Country in @vaca_country")
#reduce list to desired weather
vaca_city_df = vaca_city_df[(vaca_city_df['Humidity (%)'] < 80) & (vaca_city_df['Wind Speed (mph)'] < 20) & (vaca_city_df['Cloudiness (%)'] < 50)
                            & (vaca_city_df['Max Temperature (F)'] > 73) & (vaca_city_df['Max Temperature (F)'] < 95)]

vaca_city_df


,City Name,City ID,Country,Latitude,Longitude,Date of Data Calculation,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
8,Atuona,4020109,PF,-9.80,-139.03,1593039830,79.32,70,30,14.23
64,Road Town,3577430,VG,18.42,-64.62,1593039609,84.20,74,40,10.29
73,Olinda,3393536,BR,-8.01,-34.86,1593039835,78.80,69,20,8.05
138,San Patricio,3985168,MX,19.22,-104.70,1593039840,86.00,74,40,9.17
172,Cabo San Lucas,3985710,MX,22.89,-109.91,1593039660,93.20,12,5,19.46
187,Mareeba,2158767,AU,-17.00,145.43,1593039845,75.00,78,10,5.01
286,Lázaro Cárdenas,4026082,MX,17.96,-102.20,1593039556,82.45,77,9,15.08
329,Marabá,3395503,BR,-5.37,-49.12,1593039731,80.60,78,20,1.59
345,Puertollano,2512177,ES,38.69,-4.11,1593039858,78.01,36,1,5.10
360,Aripuanã,3665202,BR,-9.17,-60.63,1593039640,78.24,59,1,0.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create new dataframe hotel_df and add "Hotel Name" column
hotel_df = vaca_city_df
hotel_df["Hotel Name"] = ""

#set parameters for hotel search using Google Maps Places
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# using lat/lng and static params - pull hotel name using API request
for index, row in hotel_df.iterrows():   
    lat = row["Latitude"]
    lng = row["Longitude"]
    # add lat and lng to params
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    # convert to json
    hotel_name = hotel_name.json()
    # add Hotel name to hotel_df and if data is missing
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel here"
        
hotel_df


,City Name,City ID,Country,Latitude,Longitude,Date of Data Calculation,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
8,Atuona,4020109,PF,-9.80,-139.03,1593039830,79.32,70,30,14.23,Villa Enata
64,Road Town,3577430,VG,18.42,-64.62,1593039609,84.20,74,40,10.29,"ZINGARA 76ft Catamaran, full crew-all included"
73,Olinda,3393536,BR,-8.01,-34.86,1593039835,78.80,69,20,8.05,Hotel 5 Sóis
138,San Patricio,3985168,MX,19.22,-104.70,1593039840,86.00,74,40,9.17,Hotel Cabo Blanco
172,Cabo San Lucas,3985710,MX,22.89,-109.91,1593039660,93.20,12,5,19.46,Hotel Tesoro Los Cabos
187,Mareeba,2158767,AU,-17.00,145.43,1593039845,75.00,78,10,5.01,Trinity Plains Tourist Park
286,Lázaro Cárdenas,4026082,MX,17.96,-102.20,1593039556,82.45,77,9,15.08,Hotel Quinta Antigua
329,Marabá,3395503,BR,-5.37,-49.12,1593039731,80.60,78,20,1.59,Hotel Del Principe
345,Puertollano,2512177,ES,38.69,-4.11,1593039858,78.01,36,1,5.10,Hotel Puertollano
360,Aripuanã,3665202,BR,-9.17,-60.63,1593039640,78.24,59,1,0.43,No hotel here


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# add hotel markers to the map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity_fig.add_layer(markers)
humidity_fig


Figure(layout=FigureLayout(height='420px'))